In [1]:
# load data and metadata from saved files, combine them, and display unique labels
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt

# from glitchstream.glitch_downloader import GlitchDownloader
# from glitchstream.deepextractor import DeepExtractor

base_dir = pathlib.Path('./sample_glitches')

data_subs = []
for i in range(1):
	data_subs.append(np.load(base_dir/f'random_glitches_{i}_g_hats.npy',allow_pickle=True))
data = np.concatenate(data_subs,axis=0)

print(data.shape)

metadatas = []
for i in range(1):
	metadatas.append(pd.read_csv(base_dir/f'random_glitches_{i}_metadataframe.csv'))
metadata_df = pd.DataFrame(pd.concat(metadatas,ignore_index=True))

ghat_labels = metadata_df['ml_label'].to_numpy()
ghat_labels_uq = np.unique(ghat_labels)
print(np.unique(metadata_df['ml_label'].to_numpy()))

ghat_df = pd.DataFrame({'ghat':list(data),'label':ghat_labels})

ghat_df.head()

(2000, 8192)
['Blip' 'Koi_Fish' 'Scattered_Light' 'Tomte']


,ghat,label
0,"[-0.6649329863064288, -0.013300431395350643, -...",Tomte
1,"[-1.5218259913611227, 2.1265724069141783, 1.36...",Scattered_Light
2,"[1.0539129622272867, -0.062193694494517615, 3....",Blip
3,"[2.8798052331645216, 1.2827490522864409, 0.380...",Tomte
4,"[3.6875115093197017, -0.8832482481496595, 3.89...",Scattered_Light


In [ ]:
# half-supervised kshape clustering, centroid initialization consist of known labels + randomly selected samples

from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
from tslearn.clustering import KShape
from tslearn.metrics import cdist_normalized_cc

np.random.seed(42)

df_train, df_test = train_test_split(ghat_df, train_size=0.8, random_state=42,stratify=ghat_df['label'],shuffle=True)

ghat_train = np.array(df_train.ghat.to_list())
label_train = np.array(df_train.label.to_list())
ghat_test = np.array(df_test.ghat.to_list())
label_test = np.array(df_test.label.to_list())


X_train_ts = to_time_series_dataset(ghat_train)
X_test_ts = to_time_series_dataset(ghat_test)

# resample timeseries to smaller size for faster computation
sz_resample = 2048
X_train_ts_resampled = TimeSeriesResampler(sz=sz_resample).fit_transform(X_train_ts)
X_train_ts_scaled = TimeSeriesScalerMeanVariance().fit_transform(X_train_ts_resampled)

k_range = range(3,10) # number of clusters to try 3,4,5,6,7,8,9
inertia_list = []
ARI_list = []

known_centroids = []

ks_half_supervised_list = []
y_pred_ks_half_supervised_list = []
init_centroids_list = []

for i in range(len(ghat_labels_uq)):
    X_subset = X_train_ts_scaled[label_train == ghat_labels_uq[i]]
    ks_subset = KShape(n_clusters=1,random_state=42,init='random')
    ks_subset.fit(X_subset)
    known_centroids.append(ks_subset.cluster_centers_[0]) # get centroid for each known labeled class

# define a function to pick random samples as initial centroids for unknown clusters, we need to make sure their distances from
# known centroids are large enough than certain threshold (to avoid overlapping clusters)
def sbd_distance_matrix(X,Y):
    X_norms = np.linalg.norm(X,axis=(1,2))
    Y_norms = np.linalg.norm(Y,axis=(1,2))
    ncc_matrix = cdist_normalized_cc(X,Y,norms1=X_norms,norms2=Y_norms,self_similarity=False)
    return 1. - ncc_matrix

def find_safe_random_sample(X,known_centroids,random_centroids,threshold=0.75):  # input: timeseries dataset, known centroids list, random centroids list, dissimilarity threshold
    max_tries = 1000
    if len(random_centroids) > 0:
        known_centroids = known_centroids + random_centroids # exclude both known and already selected random centroids
    for _ in range(max_tries):
        rand_idx = np.random.randint(X.shape[0])
        candidate = X[rand_idx].reshape(1,-1,1)
        dists = sbd_distance_matrix(candidate,np.array(known_centroids))
        if np.min(dists**2) >= threshold:
            new_centroid = candidate.reshape(sz_resample,1)
            return new_centroid
    raise ValueError("Could not find a safe random sample after maximum tries") # if not found within max tries, raise error and abort




for k in k_range:
    if k < len(ghat_labels_uq):
        ks_half_supervised = KShape(n_clusters=k,random_state=42,init='random',n_init=10) # if k less than known clusters, random initialize
        init_centroids = 'random_initialization'
    else:
        n_random = k - len(ghat_labels_uq) # number of unknown clusters to add
        random_centroids = []
        for j in range(n_random): # begin from known centroids, add random centroids one by one
            rand_cent = find_safe_random_sample(X_train_ts_scaled,known_centroids,random_centroids)
            random_centroids.append(rand_cent)
        init_centroids = np.array(known_centroids + random_centroids)
        ks_half_supervised = KShape(n_clusters=k,random_state=42,init=init_centroids)
    y_pred_ks_half_supervised = ks_half_supervised.fit_predict(X_train_ts_scaled)
    ks_half_supervised_list.append(ks_half_supervised)
    y_pred_ks_half_supervised_list.append(y_pred_ks_half_supervised)
    init_centroids_list.append(init_centroids)
    inertia_list.append(ks_half_supervised.inertia_)
    ARI_list.append(adjusted_rand_score(label_train,y_pred_ks_half_supervised)) # compare predicted labels with known labels



In [ ]:
# plot inertia and ARI vs number of clusters k for evaluation of clustering performance

fig, ax1 = plt.figure(figsize=(8,6)), plt.gca()

ax1.set_xlabel('Number of clusters k')
ax1.set_ylabel('Inertia', color='tab:blue')
ax1.plot(k_range, inertia_list, marker='o', color='tab:blue', label='Inertia')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Adjusted Rand Index (ARI)', color='tab:red')
ax2.plot(k_range, ARI_list, marker='s', color='tab:red', label='ARI')
ax2.tick_params(axis='y', labelcolor='tab:red')

fig.tight_layout()
plt.title('Inertia and Adjusted Rand Index (ARI) vs Number of Clusters k')
plt.show() 

In [ ]:
# drawing function for visualizing cluster centroids and collection of time series in each cluster
def plot_clusters(ks_model, X_ts, y_pred, k):
    n_clusters = ks_model.n_clusters
    fig, axes = plt.subplots(n_clusters, 1, figsize=(12, 3 * n_clusters))
    if n_clusters == 1:
        axes = [axes]
    for cluster_idx in range(n_clusters):
        ax = axes[cluster_idx]
        cluster_members = X_ts[y_pred == cluster_idx]
        for ts in cluster_members:
            ax.plot(ts.ravel(), color='gray', alpha=0.2)
        centroid = ks_model.cluster_centers_[cluster_idx]
        ax.plot(centroid.ravel(), color='red', linewidth=2)
        ax.set_title(f'Cluster {cluster_idx + 1} (k={k}) - Size: {cluster_members.shape[0]}')
    plt.tight_layout()
    plt.show()

In [ ]:
# draw cluster centeroids for kbest from elbow method and ARI

kbest = 7 # kbest based on elbow method and ARI
ks_best = ks_half_supervised_list[kbest - min(k_range)]
y_pred_ks_best = y_pred_ks_half_supervised_list[kbest - min(k_range)]
label_ks_best = np.unique(y_pred_ks_best)

plot_clusters(ks_best, X_train_ts_scaled, y_pred_ks_best, kbest)